# Description

# Approach I Recursion Approach
## 太难了，不会~

In [21]:
def isMatch(text, reg):
    '''
    :type target: str
    :type reg: str
    :rtype：bool
    '''
    # if reg is empty, only return true when text is empty as well
    if not reg:
        return not text
    first_match = reg[0] in [text[0], '.'] and bool(text)
    if len(reg) >= 2 and reg[1] == '*':  # note: two * cannot be next to each other
        return isMatch(text, reg[2:]) or \
               first_match and isMatch(text[1:], reg) 
           # or之前
           # if first doesn't match, we can simply delete a matching
           # or之后
           # note1: and优先级大于or
           # note2: if first already matches, then check the next char in text but keep reg unchanged
           #        because * can match zero or many chars.
    else:
        # condition1: if there is only one char in reg, then reg[1:] = ''
        # condition2: if len(reg) >= 2 and reg[1] != '*', simply move to the next char to compare
        return first_match and isMatch(text[1:], reg[1:]) 

print(isMatch('aabc', 'c*aab'))

False


# Appoarch 2 Dynamic Programming
## Top-down
Intuition

As the problem has an optimal substructure, it is natural to cache intermediate results. We ask the question dp(i, j): does text[i:] and pattern[j:] match? We can describe our answer in terms of answers to questions involving smaller strings.

Algorithm

We proceed with the same recursion as in Approach #1, except because calls will only ever be made to match(text[i:], pattern[j:]), we use dp(i, j) to handle those calls instead, saving us expensive string-building operations and allowing us to cache the intermediate results.

In [29]:
# 自顶而下，即备忘录法 memo
# note: 特别注意这里用的表格不是二维列表，而是字典dictionary的存储形式!!!
# 60 ms in 447 tests(over 97.83%)
def isMatch(text, reg):
    memo = {}
    def dp(i, j):
        if (i, j) not in memo:
            # 如果reg匹配完了，text也恰巧匹配完了，则输出True
            if j == len(reg):
                ans = i == len(text)
            else:
                # 如果 text[i]不空，则判断text[i]与reg[j]是否匹配
                first_match = i < len(text) and reg[j] in {text[i], '.'}
                
                # 匹配了第一个字符后，查看reg[j]后一位是否是 *, 为防止溢出需要判断 j+1是否小于len(reg)
                if j+1 < len(reg) and reg[j+1] == '*':
                    ans = dp(i, j+2) or first_match and dp(i+1, j)
                # or之前
                # if reg[j+1] = '*', first doesn't match, we can simply delete a matching
                # or之后
                # note1: and优先级大于or
                # note2: if first already matches, then just check the next char in text but keep reg
                #        unchanged because * can match zero or many chars.
                else:
                # condition1: if there is only one char in reg, then reg[1:] = ''
                # condition2: if len(reg) >= 2 and reg[1] != '*', simply move to the next char to compare
                    ans = first_match and dp(i+1, j+1)
            memo[i, j] = ans
#         print(memo)
        return memo[i, j]
    return dp(0, 0)

print(isMatch('aabc', 'c*aab'))

{(3, 5): False}
{(3, 5): False, (2, 4): False}
{(3, 5): False, (2, 4): False, (1, 3): False}
{(3, 5): False, (2, 4): False, (1, 3): False, (0, 2): False}
{(3, 5): False, (2, 4): False, (1, 3): False, (0, 2): False, (0, 0): False}
False


In [21]:
# Bottom-up variation
# dp(i, j): does text[i:] and pattern[j:] match?
# 112 ms in 447 tests(over 40.56%)
def isMatch(text, pattern):
    dp = [[False] * (len(pattern)+1) for _ in range(len(text)+1)]
    # 防止越界
    dp[-1][-1] = True
    for i in range(len(text), -1, -1):
    # i从len(text)开始循环，而不是len(text)-1的目的是为了处理text为空时的匹配情况
    # 实在不理解可用spyder调试
        for j in range(len(pattern) - 1, -1, -1):
            first_match = i < len(text) and pattern[j] in {text[i], '.'}
            print(i, j)
            if j+1 < len(pattern) and pattern[j+1] == '*':
                dp[i][j] = dp[i][j+2] or first_match and dp[i+1][j]
            else:
                dp[i][j] = first_match and dp[i+1][j+1]
    for c in dp:
        print(c)
    return dp[0][0]
print(isMatch('aa', 'a*'))

1 1
1 0
0 1
0 0
[False, False, False]
[False, False, False]
[False, False, True]
False


Time Complexity: Let T, P be the lengths of the text and the pattern respectively. The work for every call to dp(i, j) for i=0,...,T; j=0, ... ,P is done once, and it is O(1) work. Hence, the time complexity is O(TP).

Space Complexity: The only memory we use is the O(TP) boolean entries in our cache. Hence, the space complexity is O(TP).